In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.9 MB/s eta 0:00:00


In [4]:
import unicodedata
import itertools
import struct
import hashlib
import os
import string
from typing import Tuple
from Crypto.Cipher import ChaCha20
from Crypto.PublicKey import ECC
from Crypto.Random import get_random_bytes
import random
from Crypto.Protocol.KDF import PBKDF2

# -------------------------
# Step 1: Unicode Normalize
# -------------------------
def step1_normalize(text: str) -> str:
    """
    Normalize the input Unicode string into its Compatibility Decomposition
    form, followed by Canonical Composition (NFKC). This process ensures
    that visually and semantically equivalent characters have a unique
    representation, thereby removing ambiguities arising from different
    Unicode encodings or combining marks. This normalization is critical
    in encryption to maintain consistency before further transformations.

    Args:
        text (str): Input Unicode string to normalize.

    Returns:
        str: The normalized Unicode string in NFKC form.
    """
    return unicodedata.normalize('NFKC', text)


def step1_denormalize(text: str) -> str:
    """
    Identity function for denormalization in this scheme. Since the
    normalization applied is deterministic and lossless for this use case,
    no explicit inverse operation is needed or applied here.

    Args:
        text (str): Input string (already normalized).

    Returns:
        str: The same input string, unchanged.
    """
    return text


# -------------------------
# Step 2: Word-wise Reverse
# -------------------------
def step2_reverse_words(text: str) -> str:
    """
    Reverse each word in the input string independently while preserving
    the order of the words themselves. This obfuscation step provides
    a simple but effective scrambling of textual content, increasing
    confusion without altering whitespace or word order.

    Args:
        text (str): The normalized input string consisting of words separated by spaces.

    Returns:
        str: String where each word's characters are reversed, but word order remains unchanged.
    """
    return ' '.join(word[::-1] for word in text.split())


def step2_reverse_words_back(text: str) -> str:
    """
    Inverse operation of step2_reverse_words. Reversing each word again
    restores the original words because string reversal is an involution.

    Args:
        text (str): The word-wise reversed string.

    Returns:
        str: The original string with words restored.
    """
    return ' '.join(word[::-1] for word in text.split())


# -------------------------
# Step 3: Autokey Vigenère
# -------------------------
ALPHABET = string.printable
ALPHA_LEN = len(ALPHABET)

def step3_autokey_vigenere(text: str, key: str = "Σ1gma") -> str:
    """
    Encrypt the input text using an Autokey Vigenère cipher variant over
    the defined printable ASCII set. The key stream is extended dynamically
    by appending each plaintext character after encryption, increasing
    complexity and resistance to simple frequency analysis.

    This method operates only on characters within the ALPHABET, leaving
    others unchanged.

    Args:
        text (str): The plaintext string to encrypt.
        key (str): The initial key string seed.

    Returns:
        str: The encrypted ciphertext string.
    """
    seq, full_key = [], list(key)
    for p in text:
        if p in ALPHABET:
            while full_key and full_key[0] not in ALPHABET:
                full_key.pop(0)
            if not full_key:
                raise ValueError("Out of key characters")
            k = full_key.pop(0)
            i = (ALPHABET.index(p) + ALPHABET.index(k)) % ALPHA_LEN
            seq.append(ALPHABET[i])
            full_key.append(p)
        else:
            seq.append(p)
    return ''.join(seq)


def step3_autokey_vigenere_back(text: str, key: str = "Σ1gma") -> str:
    """
    Decrypt the Autokey Vigenère cipher by reversing the encryption
    process. It reconstructs the keystream by appending each recovered
    plaintext character dynamically, enabling accurate reversal.

    Args:
        text (str): The ciphertext string to decrypt.
        key (str): The initial key string seed.

    Returns:
        str: The decrypted plaintext string.
    """
    seq, full_key = [], list(key)
    for c in text:
        if c in ALPHABET:
            while full_key and full_key[0] not in ALPHABET:
                full_key.pop(0)
            if not full_key:
                raise ValueError("Out of key characters")
            k = full_key.pop(0)
            i = (ALPHABET.index(c) - ALPHABET.index(k)) % ALPHA_LEN
            seq.append(ALPHABET[i])
            full_key.append(ALPHABET[i])
        else:
            seq.append(c)
    return ''.join(seq)


# -------------------------
# Step 4: ChaCha20 Stream XOR
# -------------------------
def step4_chacha_xor(data: bytes, key: bytes, nonce: bytes) -> bytes:
    """
    Symmetric encryption and decryption using the ChaCha20 stream cipher.
    The function encrypts the input data by generating a pseudorandom keystream
    based on the key and nonce, which is XORed with the data. Due to the
    stream cipher's involutory property, applying this function twice with the
    same key and nonce recovers the original data.

    Args:
        data (bytes): The input plaintext or ciphertext bytes.
        key (bytes): The secret key for ChaCha20 (must be 32 bytes).
        nonce (bytes): A nonce (number used once) for ChaCha20 (usually 12 bytes).

    Returns:
        bytes: The encrypted or decrypted data.
    """
    cipher = ChaCha20.new(key=key, nonce=nonce)
    return cipher.encrypt(data)

# This function is its own inverse.
step4_chacha_xor_back = step4_chacha_xor


# -------------------------
# Step 5: 16-Round Feistel
# -------------------------
def feistel_round(L: bytes, R: bytes, subkey: bytes) -> Tuple[bytes, bytes]:
    """
    Perform one Feistel round transformation on 4-byte halves L and R
    using the provided subkey. The round function computes a SHA-256 hash
    of R concatenated with the subkey and XORs part of the hash with L.

    Args:
        L (bytes): Left half of the block (4 bytes).
        R (bytes): Right half of the block (4 bytes).
        subkey (bytes): Round subkey derived from the master key.

    Returns:
        Tuple[bytes, bytes]: The updated (L, R) pair after one Feistel round.
    """
    h = hashlib.sha256(R + subkey).digest()
    F = h[:len(R)]
    return R, bytes(a ^ b for a, b in zip(L, F))


def step5_feistel_encrypt(data: bytes, master_key: bytes) -> bytes:
    """
    Encrypt the input data using a 16-round Feistel cipher operating
    on 8-byte blocks. Each block is split into two 4-byte halves (L, R).
    Subkeys for each round are derived via SHA-256 hashing of the master
    key concatenated with the round number. Padding with zero bytes is
    applied to ensure a multiple of 8 bytes.

    Args:
        data (bytes): The plaintext bytes to encrypt.
        master_key (bytes): The secret key used to derive round subkeys.

    Returns:
        bytes: The ciphertext bytes after Feistel encryption.
    """
    pad_len = (-len(data)) % 8
    data += b'\x00' * pad_len
    out = bytearray()
    keys = [hashlib.sha256(master_key + struct.pack(">I", r)).digest() for r in range(16)]
    for i in range(0, len(data), 8):
        L, R = data[i:i+4], data[i+4:i+8]
        for rk in keys:
            L, R = feistel_round(L, R, rk)
        out.extend(R + L)
    return bytes(out)


def step5_feistel_decrypt(data: bytes, master_key: bytes) -> bytes:
    """
    Decrypt data encrypted with step5_feistel_encrypt. Processes each
    8-byte block in reverse Feistel order, using the same round keys
    in reverse. The original data is recovered by XORing with the Feistel
    function outputs. Padding zero bytes are stripped at the end.

    Args:
        data (bytes): The ciphertext bytes to decrypt.
        master_key (bytes): The secret key used to derive round subkeys.

    Returns:
        bytes: The recovered plaintext bytes, with padding removed.
    """
    out = bytearray()
    keys = [hashlib.sha256(master_key + struct.pack(">I", r)).digest() for r in range(16)]
    for i in range(0, len(data), 8):
        R_enc, L_enc = data[i:i+4], data[i+4:i+8]
        L, R = L_enc, R_enc
        for rk in reversed(keys):
            prev_R = L
            F = hashlib.sha256(prev_R + rk).digest()[:len(prev_R)]
            prev_L = bytes(a ^ b for a, b in zip(R, F))
            L, R = prev_L, prev_R
        out.extend(L + R)
    return bytes(out).rstrip(b'\x00')


# -------------------------
# Step 6: Byte Permutation
# -------------------------
def step6_permute_bytes(data: bytes, master_key: bytes) -> bytes:
    """
    Applies a cryptographically deterministic permutation to the input byte sequence.

    The permutation is derived from a pseudorandom generator seeded by a hash of the master key,
    ensuring reproducibility for reversible transformations. This step contributes to diffusion
    by rearranging byte positions, complicating statistical attacks that exploit byte locality.

    Args:
        data (bytes): Input byte sequence.
        master_key (bytes): Key used to seed the permutation.

    Returns:
        bytes: Permuted byte sequence.
    """
    import random, hashlib
    seed = int.from_bytes(hashlib.sha256(master_key).digest()[:8], 'big')
    rnd = random.Random(seed)
    perm = list(range(len(data)))
    rnd.shuffle(perm)
    return bytes(data[i] for i in perm)


def step6_unpermute_bytes(data: bytes, master_key: bytes) -> bytes:
    """
    Inverts the byte permutation defined in step6_permute_bytes.

    Args:
        data (bytes): Permuted byte sequence.
        master_key (bytes): Key used to reconstruct the permutation.

    Returns:
        bytes: Original byte sequence.
    """
    import random, hashlib
    n = len(data)
    seed = int.from_bytes(hashlib.sha256(master_key).digest()[:8], 'big')
    rnd = random.Random(seed)
    perm = list(range(n))
    rnd.shuffle(perm)
    inv = [0]*n
    for i, p in enumerate(perm):
        inv[p] = i
    return bytes(data[inv[i]] for i in range(n))


# -------------------------
# Step 7: Bitwise Rotation
# -------------------------
def step7_rotate(data: bytes) -> bytes:
    """
    Executes a position-dependent bitwise left rotation on each byte.

    Args:
        data (bytes): Input byte sequence.

    Returns:
        bytes: Rotated byte sequence.
    """
    return bytes(((b << ((i*3)%8)) & 0xFF) | (b >> (8-((i*3)%8)))
                 for i, b in enumerate(data))


def step7_unrotate(data: bytes) -> bytes:
    """
    Reverses the bitwise rotation performed in step7_rotate.

    Args:
        data (bytes): Rotated byte sequence.

    Returns:
        bytes: Original byte sequence.
    """
    return bytes((b >> ((i*3)%8)) | ((b << (8-((i*3)%8))) & 0xFF)
                 for i, b in enumerate(data))


# -------------------------
# Step 8: SHA-512 Keystream XOR
# -------------------------
def step8_xor_sha512(data: bytes, master_key: bytes) -> bytes:
    """
    XORs input data with SHA-512 derived keystream blocks.

    Args:
        data (bytes): Input byte sequence.
        master_key (bytes): Key for generating keystream.

    Returns:
        bytes: XORed output sequence.
    """
    import hashlib
    out = bytearray()
    ctr = 0
    i = 0
    while i < len(data):
        block = data[i:i+64]
        h = hashlib.sha512(master_key + ctr.to_bytes(4,'big')).digest()
        out.extend(b ^ h[j] for j, b in enumerate(block))
        i += 64
        ctr += 1
    return bytes(out)

step8_xor_sha512_back = step8_xor_sha512


# -------------------------
# Step 9: Modular Exponentiation
# -------------------------
def step9_modexp(data: bytes, e: int, n: int) -> bytes:
    """
    Applies modular exponentiation to each 4-byte segment.

    Args:
        data (bytes): Input data in bytes.
        e (int): Public exponent.
        n (int): Modulus.

    Returns:
        bytes: Result of modular exponentiation.
    """
    out = bytearray()
    for i in range(0, len(data), 4):
        v = int.from_bytes(data[i:i+4], 'big')
        out.extend(pow(v, e, n).to_bytes(4, 'big'))
    return bytes(out)


def step9_modexp_back(data: bytes, d: int, n: int) -> bytes:
    """
    Reverses modular exponentiation using private exponent.

    Args:
        data (bytes): Encrypted byte data.
        d (int): Private exponent.
        n (int): Modulus.

    Returns:
        bytes: Decrypted byte data.
    """
    out = bytearray()
    for i in range(0, len(data), 4):
        v = int.from_bytes(data[i:i+4], 'big')
        out.extend(pow(v, d, n).to_bytes(4, 'big'))
    return bytes(out)


# -------------------------
# Step 10: XOR-Salt Only
# -------------------------
def step10_salt_encrypt(data: bytes) -> Tuple[bytes, bytes]:
    """
    XORs input with a cyclic 16-byte random salt.

    Args:
        data (bytes): Input byte data.

    Returns:
        Tuple[bytes, bytes]: XORed data and the salt used.
    """
    salt = get_random_bytes(16)
    return bytes(a ^ b for a, b in zip(data, itertools.cycle(salt))), salt


def step10_salt_decrypt(data: bytes, salt: bytes) -> bytes:
    """
    Reverses salt-based XOR encryption.

    Args:
        data (bytes): XORed byte data.
        salt (bytes): The salt used during encryption.

    Returns:
        bytes: Original byte data.
    """
    return bytes(a ^ b for a, b in zip(data, itertools.cycle(salt)))


# -------------------------
# Step 11: Key-derived S-box
# -------------------------
def step11_sbox_subst(data: bytes, master_key: bytes) -> bytes:
    """
    Applies S-box substitution based on master key.

    Args:
        data (bytes): Input byte sequence.
        master_key (bytes): Key used to generate the S-box.

    Returns:
        bytes: S-box substituted data.
    """
    import random, hashlib
    box = list(range(256))
    seed = int.from_bytes(hashlib.sha256(master_key).digest()[:8], 'big')
    rnd = random.Random(seed)
    rnd.shuffle(box)
    return bytes(box[b] for b in data)


def step11_sbox_subst_back(data: bytes, master_key: bytes) -> bytes:
    """
    Reverses S-box substitution from step11_sbox_subst.

    Args:
        data (bytes): S-box substituted byte data.
        master_key (bytes): Key used to generate inverse S-box.

    Returns:
        bytes: Original byte sequence.
    """
    import random, hashlib
    box = list(range(256))
    seed = int.from_bytes(hashlib.sha256(master_key).digest()[:8], 'big')
    rnd = random.Random(seed)
    rnd.shuffle(box)
    inv = [0]*256
    for i, v in enumerate(box):
        inv[v] = i
    return bytes(inv[b] for b in data)


# -------------------------
# Step 12: Block Shuffle
# -------------------------
def step12_block_shuffle(data: bytes, block_size: int = 16) -> bytes:
    """
    Reverses the order of fixed-size blocks in the byte stream.

    Args:
        data (bytes): Input byte sequence.
        block_size (int): Size of each block.

    Returns:
        bytes: Block-shuffled byte sequence.
    """
    full_blocks = [data[i:i+block_size]
                   for i in range(0, len(data) // block_size * block_size, block_size)]
    tail = data[len(full_blocks)*block_size:]
    full_blocks.reverse()
    return b''.join(full_blocks) + tail

step12_block_shuffle_back = step12_block_shuffle


# -------------------------
# Step 13: Nibble-Swap
# -------------------------
def step13_nibble_swap(data: bytes) -> bytes:
    """
    Swaps high and low nibbles in each byte.

    Args:
        data (bytes): Input byte sequence.

    Returns:
        bytes: Nibble-swapped output.
    """
    return bytes(((b & 0x0F) << 4) | ((b & 0xF0) >> 4) for b in data)

step13_nibble_swap_back = step13_nibble_swap


# -------------------------
# Encode / Decode
# -------------------------
def encode(plaintext: str, master_key: bytes, nonce: bytes) -> Tuple[bytes, bytes]:
    """
    Applies all 13 encryption steps on input plaintext.

    Args:
        plaintext (str): The input message to encrypt.
        master_key (bytes): Master key used in key-dependent steps.
        nonce (bytes): Nonce used in ChaCha20 encryption.

    Returns:
        Tuple[bytes, bytes]: Final ciphertext and salt used.
    """
    t = step1_normalize(plaintext)
    t = step2_reverse_words(t)
    t = step3_autokey_vigenere(t)
    b = step4_chacha_xor(t.encode('utf-8'), master_key, nonce)
    b = step5_feistel_encrypt(b, master_key)
    b = step6_permute_bytes(b, master_key)
    b = step7_rotate(b)
    b, salt = step10_salt_encrypt(b)
    b = step8_xor_sha512(b, master_key)
    b = step11_sbox_subst(b, master_key)
    b = step12_block_shuffle(b)
    b = step13_nibble_swap(b)
    return b, salt


def decode(ciphertext: bytes, master_key: bytes, nonce: bytes, salt: bytes) -> str:
    """
    Reverses the 13-layer encryption process on ciphertext.

    Args:
        ciphertext (bytes): Encrypted byte data.
        master_key (bytes): Master key used in encryption.
        nonce (bytes): Nonce used in ChaCha20.
        salt (bytes): Salt used during encryption.

    Returns:
        str: Recovered plaintext string.
    """
    b = step13_nibble_swap_back(ciphertext)
    b = step12_block_shuffle_back(b)
    b = step11_sbox_subst_back(b, master_key)
    b = step8_xor_sha512_back(b, master_key)
    b = step10_salt_decrypt(b, salt)
    b = step7_unrotate(b)
    b = step6_unpermute_bytes(b, master_key)
    b = step5_feistel_decrypt(b, master_key)
    b = step4_chacha_xor_back(b, master_key, nonce)
    t = b.decode('utf-8')
    t = step3_autokey_vigenere_back(t)
    t = step2_reverse_words_back(t)
    return step1_denormalize(t)


# ---------------------------------------
# Compact / Expand Helpers
# ---------------------------------------
BASE62_ALPHABET = string.digits + string.ascii_uppercase + string.ascii_lowercase

def super_compact_all(salt: bytes, ct: bytes) -> str:
    """
    Serializes and encodes the concatenation of the salt and ciphertext into a compact
    single string using a Base62 alphabet, optimizing for alphanumeric representation.
    This encoding facilitates safe and efficient textual transport of binary data.

    Args:
        salt (bytes): A 16-byte random salt used during encryption.
        ct (bytes): The ciphertext output from the encryption process.

    Returns:
        str: A Base62-encoded string that compactly represents the salt and ciphertext.
    """
    combined = salt + ct
    num = int.from_bytes(combined, 'big')
    if num == 0:
        return BASE62_ALPHABET[0]
    s = ''
    base = len(BASE62_ALPHABET)
    while num:
        num, rem = divmod(num, base)
        s = BASE62_ALPHABET[rem] + s
    return s

def super_expand_all(compact_str: str) -> Tuple[bytes, bytes]:
    """
    Decodes a Base62-encoded string back into the original salt and ciphertext bytes,
    enabling correct extraction of cryptographic parameters required for decryption.
    This function inverses the super_compact_all encoding procedure.

    Args:
        compact_str (str): A Base62-encoded string representing the concatenated salt and ciphertext.

    Returns:
        Tuple[bytes, bytes]: A tuple containing:
            - salt (bytes): The 16-byte salt used during encryption.
            - ct (bytes): The original ciphertext before compacting.
    """
    num = 0
    base = len(BASE62_ALPHABET)
    for c in compact_str:
        num = num * base + BASE62_ALPHABET.index(c)
    b = num.to_bytes((num.bit_length() + 7) // 8, 'big')
    salt, ct = b[:16], b[16:]
    return salt, ct


# ---------------------------------------
# CLI Interface
# ---------------------------------------

if __name__ == "__main__":
    """
    Command-line interface for the 13-layer encryption system.

    This interface requires the user to enter a passphrase (which is the reverse of the word "passphrase").
    Once authenticated, the user can choose to encode (encrypt) or decode (decrypt) messages using a
    unified, compact Base62 format for convenient input/output.

    Features:
        - Secure PBKDF2-derived master key and nonce generation.
        - User prompt for encoding (plaintext input) or decoding (Base62 string input).
        - Reversible compact serialization using Base62.
    """

    PASS = input("Enter passphrase: ")
    if PASS != "esarhpssap":
        print("Access denied: incorrect passphrase.")
        mode = 'q'
    else:
        salt_for_kdf = b'\x00\x11\x22\x33\x44\x55\x66\x77\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
        MASTER_KEY = PBKDF2(PASS, salt_for_kdf, dkLen=32, count=100000)
        NONCE = MASTER_KEY[:12]

        print("Type 'e' to encode, 'd' to decode, or 'q' to quit.")
        mode = ''

    while mode != 'q':
        mode = input("\nYour choice (e/d/q): ").strip().lower()

        if mode == 'q':
            break

        elif mode == 'e':
            pt = input("Enter plaintext: ")
            ct, salt = encode(pt, MASTER_KEY, NONCE)
            unified = super_compact_all(salt, ct)
            print("\nSuper-Compact (one-field):", unified)

        elif mode == 'd':
            try:
                unified = input("Paste Super-Compact string: ").strip()
                salt, ct = super_expand_all(unified)
                pt = decode(ct, MASTER_KEY, NONCE, salt)
                print("\nPlaintext:", pt)
            except Exception as e:
                print("Decoding failed:", e)

        else:
            print("Invalid; enter e, d, or q.")


Enter passphrase: esarhpssap
Type 'e' to encode, 'd' to decode, or 'q' to quit.

Your choice (e/d/q): e
Enter plaintext: I have been reflecting on the recent lectures regarding quantum mechanics and its fundamental principles.

Super-Compact (one-field): FrsQCxd6d2FOleZ1JSS9C5aAq55TARGxGY42cTS3VDsbIQNiwQhSQDSS0dfnzsO750QYFkEf1w1q3VdX2YRKijOmtCudBJGuFicIo3xXx12YSEI8Z9AmwXmHpJOYiUrUWDjtgrPRdTK6CFNN118GMTGwkzdNvoF6RS8awSQbJjqy

Your choice (e/d/q): d
Paste Super-Compact string: FrsQCxd6d2FOleZ1JSS9C5aAq55TARGxGY42cTS3VDsbIQNiwQhSQDSS0dfnzsO750QYFkEf1w1q3VdX2YRKijOmtCudBJGuFicIo3xXx12YSEI8Z9AmwXmHpJOYiUrUWDjtgrPRdTK6CFNN118GMTGwkzdNvoF6RS8awSQbJjqy

Plaintext: I have been reflecting on the recent lectures regarding quantum mechanics and its fundamental principles.

Your choice (e/d/q): q
